In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 26.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
     |████████████████████████████████| 596 kB 33.1 MB/s 
     |████████████████████████████████| 880 kB 61.3 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9b567cdd9fd115e03c52a70881eaa6bc3ce9653803fe4a08290185bb3a8b7164
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [22]:
df = pd.read_csv('/content/KB_censored-lyrics.csv')
df.head()

,ogArtist,songName,badword,count,category,year,isCensored,isPresent,ogLyric,kbLyric
0,Ariana Grande,7 rings,bitch,2,profanity,2019,1,1,Bought matching diamonds for six of my bitches,Bought matching diamonds so they know what thi...
1,Britney Spears,till the world ends,blow,1,alcohol & drugs,2011,1,1,"Baby, let me blow your mind tonight","Baby, come on wave your hands tonight"
2,Britney Spears,till the world ends,blow,2,alcohol & drugs,2011,1,1,"Baby, let me blow your mind tonight","Baby, come on wave your hands tonight"
3,Britney Spears,till the world ends,touch,1,sexual,2011,1,1,"Get you off with the touch, dancing in the dark",Set it off with a track dancing in the dark
4,Elle King,ex's and oh's,high,1,alcohol & drugs,2016,1,1,"I get high, and I love to get low","I get by, and I love to lay low"


In [23]:
len(df)

1353

In [28]:
class processedData(Dataset):
    
    def __init__(self, inputData):

        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.lyrics = []
        for row in df['ogLyric']:
          self.lyrics.append(torch.tensor( self.tokenizer.encode(f"<|{inputData}|>{row[:1024]}<|endoftext|>")))
          
        self.lyrics = self.lyrics[:20000]

        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [29]:
dataset = processedData(df['ogLyric'])

In [30]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [31]:
def train( dataset, model, tokenizer, gpt2_type="gpt2", b_s=16, epochs=20, lr=2e-5, max_seq_len=400,):
    loss=0
    abc = 0
    model = model.cuda()
    model.train()

    opt = AdamW(model.parameters(), lr=lr)
    sch = get_linear_schedule_with_warmup( opt, num_warmup_steps=200, num_training_steps=-1)
    input_tensor = None

    for ep in range(epochs):
        print(f"Training epoch {ep}")
        print(loss)
        ind = 0
        for entry in tqdm(DataLoader(dataset, batch_size=1, shuffle=True)):
            if input_tensor is None:
                input_tensor = entry
                carry_on = True
                remainder = None
                if ind != len(DataLoader(dataset, batch_size=1, shuffle=True))-1:
                    continue
            elif entry.size()[1] + input_tensor.size()[1] > 768:
                input_tensor = input_tensor
                carry_on = False
                remainder = entry
            else:
                input_tensor = torch.cat([entry, input_tensor[:, 1:]], dim=1)
                carry_on = True
                remainder = None
                if ind != len(DataLoader(dataset, batch_size=1, shuffle=True)) - 1:
                    continue

            input_tensor = input_tensor.to(torch.device("cuda"))
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (abc % b_s) == 0:
                opt.step()
                sch.step()
                opt.zero_grad()
                model.zero_grad()

            abc += 1
            input_tensor = None
            ind += 1
    return model

In [21]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


1303it [03:13,  6.73it/s]


Training epoch 1
tensor(1.1573, device='cuda:0', grad_fn=<NllLossBackward0>)


1303it [03:14,  6.68it/s]


...

1303it [03:13,  6.72it/s]


Training epoch 18
tensor(0.2623, device='cuda:0', grad_fn=<NllLossBackward0>)


1303it [03:14,  6.71it/s]


Training epoch 19
tensor(0.1754, device='cuda:0', grad_fn=<NllLossBackward0>)


1303it [03:14,  6.69it/s]


In [26]:
text = generate(model.to('cpu'), tokenizer, "I love deep Learning", entry_count=1)

100%|██████████| 1/1 [00:05<00:00,  5.82s/it]


In [27]:
print(text)

["I love deep Learning, but it's not cheap. The price of today's deep learning is $30 and you just have to know to spend it. It's only<|endoftext|>"]
